In [1]:
# 这部分是模板，用于 Jupyter lab 文件的初始导入和设置。
# 1. import built-in library
from copy import deepcopy
from datetime import datetime
import functools
import json
import math
import os
from random import shuffle
import sys
import time

# 2. import 3rd party library
import cv2 as cv
from IPython.core.interactiveshell import InteractiveShell
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import PIL
from PIL import ImageOps
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

# 3. 导入自定义的函数
sys.path.append('D:\deep_learning\computer_vision')  # 导入 plot_utils 的存放路径
import plot_utils  

rng = np.random.default_rng()

# setup for the jupyter to show all results within one cell
InteractiveShell.ast_node_interactivity = "all"

# 实时更新导入的外部 python 程序
%load_ext autoreload
%autoreload 2

# 控制显存.
config=tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  # 设置动态分配 GPU 内存
sess=tf.compat.v1.Session(config=config)

tf.config.list_physical_devices('GPU')

# YOLO-V4-CSP模型用到 math.atan ，因为 TF 2.4 不支持使用 float16 计算 math.atan， 所以在TF 2.4 中不使用混合精度 mixed precision。在 TF 2.8 中支持使用 float16 计算 math.atan，已经可以用混合精度进行加速。
# 但是因为在损失函数中，计算数值超过 float16 表达范围，所以 YOLO-V4-CSP 依然无法使用混合精度加速。

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
tf.__version__

'2.9.1'

In [3]:
from create_tf_dataset import coco_data_yolov4_csp
from create_tf_dataset import CATEGORIES_TO_DETECT

import yolo_v4_csp
from yolo_v4_csp import visualize_predictions
# visualize_predictions?

Extracting the annotations for train dataset ...
851121/860001 [==========================================================>.] - ETA: 0s
Done. Here are 2 fixed records.
Height was 0, set to 1. 	Image: 200365	category_id: 58,	annotation index: 3,	object center 297.2, 388.3, 
Height was 0, set to 1. 	Image: 550395	category_id: 1,	annotation index: 846309,	object center 12.8, 188.6, 
Extracting the annotations for validation dataset ...
34401/36781 [=======================================================>....] - ETA: 0s

In [4]:
batch_size = 8
# 创建训练集和验证集。
validation_dataset = coco_data_yolov4_csp(
    dataset_type='validation', images_range=(0, 500), batch_size=batch_size)  
validation_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(8, 608, 608, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(8, 19, 19, 3, 85), dtype=tf.float32, name=None), TensorSpec(shape=(8, 38, 38, 3, 85), dtype=tf.float32, name=None), TensorSpec(shape=(8, 76, 76, 3, 85), dtype=tf.float32, name=None)))>

In [5]:
start_image = 0
# 为了快速展现模型的过拟合能力，下面只使用 8 张图片进行演示。如果有 GPU 集群对模型进行训练，则可以使用任意数量的图片。
images_quantity = 8 
train_dataset = coco_data_yolov4_csp(
    dataset_type='train', images_range=(start_image, start_image+images_quantity),
    shuffle_images=False, batch_size=batch_size)  

train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(8, 608, 608, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(8, 19, 19, 3, 85), dtype=tf.float32, name=None), TensorSpec(shape=(8, 38, 38, 3, 85), dtype=tf.float32, name=None), TensorSpec(shape=(8, 76, 76, 3, 85), dtype=tf.float32, name=None)))>

In [6]:
# 设置 2 个路径，用在自动保存模型的 callbacks 中。要保存 2 个模型，一个是最高 AP 值模型，另外一个是持续训练模型（即每个 epoch 结束后就保存一次的模型）。
ongoing_training_model_name = 'ongoing_training_yolo_v4_csp'
ongoing_training_model_path = f'checkpoints/{ongoing_training_model_name}.keras'  

highest_ap_model_name = 'highest_ap_yolo_v4_csp'
highest_ap_model_path = f'checkpoints/{highest_ap_model_name}.keras'

In [7]:
yolo_v4_csp_model = yolo_v4_csp.create_model()
# 画出模型结构图。
# keras.utils.plot_model(model=yolo_v4_csp_model,
#                        to_file='yolo_v4_csp_model.png', show_shapes=True, dpi=120)

In [8]:
# 使用学习率阶梯衰减 step decay。
def scheduler(epoch, lr):
    """Step learning rate decay."""
    
    if epoch >= epochs_first_lr_decay and (epoch - epochs_first_lr_decay) % epochs_second_lr_decay == 0:
        # 在若干个迭代之后，对学习率进行阶跃衰减。建议同时打印信息，对用户进行提示。
        print(f'Changing the learning rate, \nbefore change: {lr:.2e}')
        lr *= rate_lr_decay
        print(f'after change: {lr:.2e}')   
    return lr
    
lr_decay_callback = keras.callbacks.LearningRateScheduler(scheduler)

In [9]:
# 创建一个空的表格，用于后续记录训练数据。counter_records 用于计数，必须和表格同时初始化。
records = pd.DataFrame({})
counter_records = 0

In [10]:
tic = time.time()

# 以下 3 个设置为学习率衰减设置，可以进行 2 次衰减。
# 阶梯衰减也可以用 keras.optimizers.schedules.PiecewiseConstantDecay
epochs_first_lr_decay = 7000  # 10
epochs_second_lr_decay = 20000
rate_lr_decay = 0.1 

# numbers = 4  # 根据快速排序算法，应该每次使用 4 个数据点，才能尽快找到最佳超参。
# learning_rates = np.logspace(np.log10(1e-1), np.log10(1e-3), num=numbers)
# learning_rates = np.linspace(3e-2, 1.5e-2, num=numbers)  # 0.1
# learning_rates = [8e-04]  
learning_rate = 8e-04   

# weights_classification = np.logspace(np.log10(100), np.log10(0.1), num=numbers)
# weights_classification = np.linspace(130, 55, num=numbers)
weights_classification = [10]  # 100, 50, 20, 10, 1  
# weight_classification = 10  #10

# weights_ciou = np.logspace(np.log10(10), np.log10(0.01), num=numbers)  # 0.04
# weights_ciou = np.linspace(np.log10(10), np.log10(1), num=numbers)
weights_ciou = [0.01] 
# weight_ciou = 0.01  

# 如果只用 8 张图片进行演示，约训练 6000 个 epochs，就可以使得 AP 达到 100%。
epochs = 10000

# 正常训练时，应该使用 validation_dataset 作为下面回调函数的 evaluation_data。
# 下面使用 8 张图片进行演示，所以用 train_dataset 作为 evaluation_data。
save_highest_ap_callback = yolo_v4_csp.SaveModelHighestAP(
    evaluation_data=train_dataset, highest_ap_model_path=highest_ap_model_path,
    epochs_warm_up=9800, skip_epochs=5,
    ongoing_training_model_path=ongoing_training_model_path)

callbacks_list = [save_highest_ap_callback, lr_decay_callback]

# ======= 此部分用于长期训练。==============================================================

# 如果是长期对模型进行训练，并且在训练多天之后停止了训练，可以用下面 2 行把保存的模型加载进来。
# custom_objects = {'MishActivation': yolo_v4_csp.MishActivation} 
# yolo_v4_csp_model = keras.models.load_model(filepath=ongoing_training_model_path, 
#                                             custom_objects=custom_objects, compile=False)

optimizer_adam = keras.optimizers.Adam(learning_rate=learning_rate)

my_custom_loss = functools.partial(
    yolo_v4_csp.my_custom_loss, 
    focal_binary_loss=True,
    categorical_classification_loss=False,
    weight_classification=10, 
    weight_ciou=0.01, 
) 
# Keras 要用到名字属性，所以这里必须定义 __name__
my_custom_loss.__name__ = 'my_custom_loss'

yolo_v4_csp_model.compile(
    loss=my_custom_loss, 
    optimizer=optimizer_adam) 

# ======= 此部分用于长期训练。==============================================================

count_down = len(weights_classification) * len(weights_ciou)


# 下面这部分用于寻找超参，所以用 for 循环遍历 weights_classification 和 weights_ciou。
for weight_classification in weights_classification: 
    for weight_ciou in weights_ciou:

# =======仅在寻找 lr 等超参时需要使用这部分代码。对每个不同的超参组合，使用一个新的 model。=============================
#         yolo_v4_csp_model = yolo_v4_csp.create_model()

#         optimizer_adam = keras.optimizers.Adam(learning_rate=learning_rate)

#         my_custom_loss = functools.partial(
#             yolo_v4_csp.my_custom_loss, 
#             focal_binary_loss=True,
#             categorical_classification_loss=False,
#             weight_classification=weight_classification, 
#             weight_ciou=weight_ciou, 
#         ) 
#         # Keras 要用到名字属性，所以这里必须定义 __name__
#         my_custom_loss.__name__ = 'my_custom_loss'

#         yolo_v4_csp_model.compile(
#             loss=my_custom_loss, 
#             optimizer=optimizer_adam) 
# =======仅在寻找 lr 等超参时需要使用这部分代码。对每个不同的超参组合，使用一个新的 model。=============================

        current_toc = time.time()
        current_duration = current_toc - tic
        current_minutes = current_duration / 60
        current_hours_spent = current_minutes / 60 

        print(f'learning_rate: {learning_rate:.2e}, count_down: {count_down}.')
        print(f'minutes_spent: {current_minutes:.0f}. hours_spent: {current_hours_spent:.1f}.')
        print(f'weight_classification: {weight_classification:.1f}, weight_ciou: {weight_ciou:.4f}.')  
        # print(f'Training images range: {start_image:.1e}---{start_image + images_quantity:.1e}') 
        count_down -= 1                

        history = yolo_v4_csp_model.fit(
            x=train_dataset, epochs=epochs,
            verbose=1,  
            callbacks=callbacks_list)   

        history_dict = history.history

        max_weight = yolo_v4_csp.check_weights(model_input=yolo_v4_csp_model)
        max_weight = round(max_weight, 1) 

        loss = history_dict['loss']
        # validation_loss = history_dict['val_loss']

        last_loss = round(loss[-1], 3)
        drop_loss = round((loss[0] - loss[-1]), 3)

        records.loc[counter_records, 'lr'] = f'{learning_rate:.2e}'

        records.loc[counter_records, 'last_loss'] = last_loss
        records.loc[counter_records, 'epochs'] = epochs
        
        records.loc[counter_records, 'weight_class'] = weight_classification
        records.loc[counter_records, 'weight_ciou'] = weight_ciou

        records.loc[counter_records, 'epochs_lr_decay'] = epochs_first_lr_decay
        records.loc[counter_records, 'max_weight'] = max_weight

        counter_records += 1
    
# 
toc = time.time()
duration = toc - tic
minutes = duration / 60
hours_spent = minutes / 60

learning_rate: 8.00e-04, count_down: 1.
minutes_spent: 0. hours_spent: 0.0.
weight_classification: 10.0, weight_ciou: 0.0100.
Epoch 1/10000
1/1 [==============================] - 41s 41s/step - loss: 12.5335 - p5_loss: 3.7023 - p4_loss: 4.8837 - p3_loss: 3.9475 - lr: 8.0000e-04
Epoch 2/10000
1/1 [==============================] - 4s 4s/step - loss: 5.6913 - p5_loss: 1.8828 - p4_loss: 1.8991 - p3_loss: 1.9093 - lr: 8.0000e-04
Epoch 3/10000
1/1 [==============================] - 4s 4s/step - loss: 5.4952 - p5_loss: 1.8240 - p4_loss: 1.8182 - p3_loss: 1.8530 - lr: 8.0000e-04
Epoch 4/10000
1/1 [==============================] - 4s 4s/step - loss: 5.2339 - p5_loss: 1.7498 - p4_loss: 1.6851 - p3_loss: 1.7991 - lr: 8.0000e-04
Epoch 5/10000
1/1 [==============================] - 4s 4s/step - loss: 4.8748 - p5_loss: 1.6488 - p4_loss: 1.5099 - p3_loss: 1.7161 - lr: 8.0000e-04
Epoch 6/10000
1/1 [==============================] - 4s 4s/step - loss: 4.7116 - p5_loss: 1.5340 - p4_loss: 1.5808 - p3_l

In [11]:
# 手动检查最大权重。过大的权重容易导致 NaN 损失。
max_weight = yolo_v4_csp.check_weights(model_input=yolo_v4_csp_model)
max_weight = round(max_weight, 1)
f'The maximum weight is: {max_weight:.1f}'


Checking the weights ...
546/566 [========================================================>...] - ETA: 0s
The status is OK, max_weight is: 3.8



'The maximum weight is: 3.8'

In [12]:
# 查看训练所用的时间。
minutes
f'hours_spent: {hours_spent:.1f}'
# AP 的记录值。
save_highest_ap_callback.ap_record

613.1217660705248

'hours_spent: 10.2'

[0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [14]:
from yolo_v4_csp import LATEST_RELATED_IMAGES, BBOXES_PER_IMAGE
pd.set_option('display.max_rows', 90)

f'LATEST_RELATED_IMAGES: {LATEST_RELATED_IMAGES}, BBOXES_PER_IMAGE: {BBOXES_PER_IMAGE}'
'使用 focal 二元交叉熵损失函数。'
records.sort_values(by='last_loss', ascending=False)[: 90]

'LATEST_RELATED_IMAGES: 10, BBOXES_PER_IMAGE: 10'

'使用 focal 二元交叉熵损失函数。'

,lr,last_loss,epochs,weight_class,weight_ciou,epochs_lr_decay,max_weight
0,8.00e-04,0.0,10000.0,10.0,0.01,7000.0,3.8


In [16]:
# 画出 loss 的折线图。

# val_loss_drop = history_dict['val_loss'][0] - history_dict['val_loss'][-1]
# loss_overfitting = history_dict['val_loss'][-1] - history_dict['loss'][-1]

current_time = datetime.now()
title = (f'{ongoing_training_model_name}, epochs: {epochs}, learning_rate: {learning_rate:.2e},<br />'
         f'last loss: {last_loss}, '
         f'duration: {minutes:.1f} minutes, {hours_spent:.1f} hours. {current_time:%Y-%m-%d, %H:%m}')

plot_utils.scatter_plotly_keras(history_dict, title, ongoing_training_model_name, plot_loss=True)

In [30]:
# 下面查看模型在训练集上的预测效果。
partial_train_dataset = train_dataset.take(1)
partial_train_dataset

<TakeDataset element_spec=(TensorSpec(shape=(8, 608, 608, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(8, 19, 19, 3, 85), dtype=tf.float32, name=None), TensorSpec(shape=(8, 38, 38, 3, 85), dtype=tf.float32, name=None), TensorSpec(shape=(8, 76, 76, 3, 85), dtype=tf.float32, name=None)))>

In [31]:
for element in partial_train_dataset:
    images = element[0]
    labels = element[1]

type(images)
images.shape
type(labels)

tensorflow.python.framework.ops.EagerTensor

TensorShape([8, 608, 608, 3])

tuple

In [32]:
result = yolo_v4_csp_model.predict(images)

one_image_p5 = result[0][0]
one_image_p4 = result[1][0]
one_image_p3 = result[2][0]
one_image_p5 = one_image_p5.reshape((*one_image_p5.shape[: 2], 3, 85))
one_image_p4 = one_image_p4.reshape((*one_image_p4.shape[: 2], 3, 85))
one_image_p3 = one_image_p3.reshape((*one_image_p3.shape[: 2], 3, 85))
f'one_image_p5 shape: {one_image_p5.shape}'
f'one_image_p4 shape: {one_image_p4.shape}'
f'one_image_p3 shape: {one_image_p3.shape}'
probability_p5 = tf.math.sigmoid(one_image_p5[..., 0])
probability_p4 = tf.math.sigmoid(one_image_p4[..., 0])
probability_p3 = tf.math.sigmoid(one_image_p3[..., 0])
classification_p5 = tf.math.sigmoid(one_image_p5[..., 1: 81])
classification_p4 = tf.math.sigmoid(one_image_p4[..., 1: 81])
classification_p3 = tf.math.sigmoid(one_image_p3[..., 1: 81])

1/1 [==============================] - 1s 1s/step


'one_image_p5 shape: (19, 19, 3, 85)'

'one_image_p4 shape: (38, 38, 3, 85)'

'one_image_p3 shape: (76, 76, 3, 85)'

In [33]:
f'weight_classification: {weight_classification}, weight_ciou: {weight_ciou},'
f'probability_p5, max: {tf.math.reduce_max(probability_p5):.3f}, min: {tf.math.reduce_min(probability_p5):.3f}'
f'probability_p4, max: {tf.math.reduce_max(probability_p4):.3f}, min: {tf.math.reduce_min(probability_p4):.3f}'
f'probability_p3, max: {tf.math.reduce_max(probability_p3):.3f}, min: {tf.math.reduce_min(probability_p3):.3f}'

confidence_threshold = 0.5
positive_p5 = probability_p5 > confidence_threshold
positive_quantity_p5 = tf.where(positive_p5)
f'confidence_threshold: {confidence_threshold},'
f'Positives quantity in p5: {len(positive_quantity_p5)}'
positive_p4 = probability_p4 > confidence_threshold
positive_quantity_p4 = tf.where(positive_p4)
f'Positives quantity in p4: {len(positive_quantity_p4)}'
positive_p3 = probability_p3 > confidence_threshold
positive_quantity_p3 = tf.where(positive_p3)
f'Positives quantity in p3: {len(positive_quantity_p3)}'

f'classification_p5, max: {tf.math.reduce_max(classification_p5):.3f}, min: {tf.math.reduce_min(classification_p5):.3f}'
f'classification_p4, max: {tf.math.reduce_max(classification_p4):.3f}, min: {tf.math.reduce_min(classification_p4):.3f}'
f'classification_p3, max: {tf.math.reduce_max(classification_p3):.3f}, min: {tf.math.reduce_min(classification_p3):.3f}'
f'epochs: {epochs}, lr_decay_epochs: {epochs_first_lr_decay}, lr: {learning_rate}'
f'images_quantity: {images_quantity}, start image: {start_image}'
f'max_weight: {max_weight:.1f}, last_loss: {last_loss}'

'weight_classification: 10, weight_ciou: 0.01,'

'probability_p5, max: 0.946, min: 0.020'

'probability_p4, max: 0.911, min: 0.013'

'probability_p3, max: 0.050, min: 0.019'

'confidence_threshold: 0.5,'

...

'classification_p4, max: 0.995, min: 0.002'

'classification_p3, max: 0.696, min: 0.227'

'epochs: 10000, lr_decay_epochs: 7000, lr: 0.0008'

'images_quantity: 8, start image: 0'

'max_weight: 3.8, last_loss: 0.0'

In [43]:
# 显示模型预测的结果。按 q 可以查看下一张图片。
object_exist_confidence_threshold = 0.5
classification_confidence_threshold = 0.5
visualize_predictions(
    image_input=images, predictions=result, 
    objectness_threshold=object_exist_confidence_threshold, 
    classification_threshold=classification_confidence_threshold,
    show_classification_confidence=False, 
    bboxes_quantity=200,
    categories_to_detect=CATEGORIES_TO_DETECT)


Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to c

In [ ]:
stop 1
# 下面这部分是实验。用于比较各种超参组合，经过 30 次实验后的 AP 均值和标准差。

In [18]:
# rd = records[['epochs', 'AP']]
# # 'max_weight_norm: 20, steps: 2000'
# # f'learning_rate: {learning_rate: .2e}'
# # 'rate_regularizer: 0.001'
rd = records.copy()
rd
# records

,lr,last_loss,epochs,weight_class,weight_ciou,epochs_lr_decay,max_weight
0,8.00e-04,0.0,10000.0,10.0,0.01,7000.0,3.8


In [ ]:
# 使用余弦衰减时，用这部分代码。
lr_decay_record = rd[rd['epochs_lr_decay'] == 2000]
lr_decay_all = lr_decay_record['AP']
# 要去掉有 NaN 值的部分，即 AP == 0 的部分。
lr_decay_ap = lr_decay_all[~np.isclose(lr_decay_all, 0)]
nan_lr_decay = lr_decay_all[np.isclose(lr_decay_all, 0)]

f'Consine 学习率衰减，{len(lr_decay_ap)} 个 AP， {len(nan_lr_decay)} 个 NaN'
lr_decay_ap
no_lr_decay_record = rd[rd['epochs_lr_decay'] == 'Restarts, α=8e-7']
no_lr_decay_all = no_lr_decay_record['AP']
# 要去掉有 NaN 值的部分，即 AP == 0 的部分。
no_lr_decay_ap = no_lr_decay_all[~np.isclose(no_lr_decay_all, 0)]
nan_no_lr_decay = no_lr_decay_all[np.isclose(no_lr_decay_all, 0)]

f'Restarts Consine 学习率衰减，{len(no_lr_decay_ap)} 个 AP, {len(nan_no_lr_decay)} 个 NaN'
no_lr_decay_ap

In [ ]:
f'超参数：weight_classification={weight_classification}, weight_ciou={weight_ciou}'
f'使用余弦衰减，训练 {epochs} 个 epochs。{len(lr_decay_ap)} 次 AP 如下：'
# lr_decay_ap
f'均值：{np.mean(lr_decay_ap):.3f}, 最大值：{np.amax(lr_decay_ap):.3f}, 最小值：{np.amin(lr_decay_ap)}, 标准差：{np.std(lr_decay_ap):.3f}'
# f'此外还有 {len(nan_lr_decay)} 个 NaN, NaN 数量的比例：{len(nan_lr_decay)/(len(lr_decay_ap) + len(nan_lr_decay)):.1%}'

print('='*80)
f'使用 Restarts 重复式余弦衰减，训练 {epochs} 个 epochs。{len(no_lr_decay_ap)}  次 AP 如下：'
f'alpha={learning_rate/1000:.1e}'
# no_lr_decay_ap
f'均值：{np.mean(no_lr_decay_ap):.3f}, 最大值：{np.amax(no_lr_decay_ap):.3f}, 最小值：{np.amin(no_lr_decay_ap)}, 标准差：{np.std(no_lr_decay_ap):.3f}'
# f'此外还有 {len(nan_no_lr_decay)} 个 NaN, NaN 数量的比例：{len(nan_no_lr_decay)/(len(no_lr_decay_ap) + len(nan_no_lr_decay)):.1%}'

In [ ]:
stop 2
# 下面这部分，是查看标签中的物体框。

In [44]:
# visualize_predictions(
#     image_input=partial_train_dataset, 
#     show_classification_confidence=False,
#     categories_to_detect=CATEGORIES_TO_DETECT)


Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to close all image windows.

Press key "q" to close all image windows.
Press key "s" to save the detected image.

Press any key to c